In [ ]:
"""
To Do:
- Switch Bollinger calculation to TA Lib
- Add RSI Indicator
- Create transaction output sentence for trade log
- Improve Bollinger Signal sophistication - capture velocity change (nice to have)
- Determine best buy/sell thresholds (BUY_THRESHOLD, SELL_THRESHOLD)
- Check if we need financials_df defined as global variable
"""

In [3]:
import websocket
import json
import pprint
import talib
import numpy as np
import config
import pandas as pd
import tkinter as tk
from talib import MA_Type
from binance.client import Client
from binance.enums import *

In [46]:
# Constants
SOCKET = "wss://stream.binance.com:9443/ws/btcusdt@miniTicker"
BOLLINGER_WINDOW = 20
BOLLINGER_WEIGHT = 1
WHALE_WEIGHT = 0
BUY_THRESHOLD = 1
SELL_THRESHOLD = -1
COOLDOWN_WINDOW = 2
BUY = 1
HOLD = 0
SELL = -1
MA_TYPE = 0 #Simple MA

In [30]:
# Global Variables
client = Client(config.API_KEY, config.API_SECRET, testnet = True)
financials_df = pd.DataFrame(columns = ['close', 'bollinger_signal', 'whale_signal', 'weighted_signal', 'trade_decision'])

In [26]:
# Creating Bollinger Signal
def get_bollinger_signal(close):
    upper, middle, lower = talib.BBANDS(close, timeperiod=BOLLINGER_WINDOW, matype=MA_TYPE)
    if close[-1] < lower[-1]:
        bollinger_long = 1
    else:
        bollinger_long = 0
    if close[-1] > upper[-1]:
        bollinger_short = -1
    else:
        bollinger_short = 0
    bollinger_signal = bollinger_long + bollinger_short
    return bollinger_signal

In [27]:
def get_weighted_signal(signal_values, signal_weights):
    signal_weighted_avg = np.average(signal_values, weights=signal_weights)
    
    if signal_weighted_avg >= BUY_THRESHOLD:
        weighted_signal = BUY
    elif signal_weighted_avg <= SELL_THRESHOLD:
        weighted_signal = SELL
    else:
        weighted_signal = HOLD

    return weighted_signal

In [48]:
def trade_decision():
    global financials_df
    weighted_signals = financials_df['weighted_signal'].iloc[-COOLDOWN_WINDOW:]
    current_signal = weighted_signal[-1]
    if current_signal == 0:
        trade_decision = 'hold'
    elif current_signal == 1 and weighted_signals = 1, "hold", "buy" ):
        trade_decision = 'buy'

NameError: name 'weighted_signals' is not defined

In [47]:
financials_df['close'].iloc[-COOLDOWN_WINDOW:]

3     60000.0
4    100000.0
Name: close, dtype: float64

In [37]:
financials_df

,close,bollinger_signal,whale_signal,weighted_signal,trade_decision
0,40000.0,0.0,0.0,0.0,NaN
1,45000.0,0.0,0.0,0.0,NaN
2,50000.0,0.0,0.0,0.0,NaN
3,60000.0,0.0,0.0,0.0,NaN
4,100000.0,0.0,0.0,0.0,NaN


In [28]:
def financials_df_builder(close):
    #this function's purpose is to append the latest information to the financials_df
    global financials_df
    signal_values = []
    signal_weights = []
    
    # Bollinger Signal and Weight
    bollinger_signal = 0
    if len(financials_df) >= BOLLINGER_WINDOW:
        bollinger_signal = get_bollinger_signal(close)
    signal_values.append(bollinger_signal)
    signal_weights.append(BOLLINGER_WEIGHT)
    
    # Whale Signal and Weight
    whale_signal = 0
    signal_values.append(whale_signal)
    signal_weights.append(WHALE_WEIGHT)

    weighted_signal = get_weighted_signal(signal_values, signal_weights)
    financials_df = financials_df.append({'close' : close, 'bollinger_signal' : bollinger_signal, 'whale_signal': whale_signal, 'weighted_signal' : weighted_signal}, ignore_index = True)

    return

In [11]:
def order(side, quantity, symbol, order_type=ORDER_TYPE_MARKET):
    try:
        print("sending order")
        order = client.create_order(symbol=symbol, side=side, type=order_type, quantity=quantity)
        #print(order)
    except Exception as e:
        #print("an exception occurred - {}".format(e))
        return False, e
        #return "an exception occurred"

    return True, order
    #return order

In [6]:
# Putting in an order
#success, order_response = order('buy', '0.001', 'BTCUSDT')

In [39]:
if financials_df['bollinger_signal'].iloc[-1] > 0:
    order('buy', '0.001', 'BTCUSDT')
elif financials_df['bollinger_signal'].iloc[-1] < 0:
    order('sell', '0.001', 'BTCUSDT')
else:
    print('hold')

sending order


In [43]:
# current BTC balance = 1.20840900
client.get_account()

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'updateTime': 1627346625501,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.20640900', 'locked': '0.00000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '101.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '2263.04887274', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

In [44]:
balance_btc = client.get_asset_balance(asset='BTC')
print(balance_btc)

{'asset': 'BTC', 'free': '1.20640900', 'locked': '0.00000000'}


In [21]:
def on_message(ws, message):
    #print('received message')
    json_message = json.loads(message)
    #pprint.pprint(json_message)
    close = json_message['c']
    #print('close price ' + close)
    # get Bollinger values
    #print('financials df length ' + len(financials_df))
    get_bollinger_signal(float(close))
    #print('bollinger signal ' + bollinger_signal)
    #financials_df = financials_df.append({'close' : float(close), 'bollinger_signal' : bollinger_signal})
    #print("close")
    #print('got bollinger signal')

In [22]:
def on_open(ws):
    print('opened connection')
def on_close(ws):
    print('closed connection')
ws = websocket.WebSocketApp(SOCKET, on_open=on_open, on_close=on_close, on_message=on_message)
ws.run_forever()